# Single iteration example con Gemini
Once you're happy with the first version of your functionality, you can build a scheduler around this to run on a regular basis or listen for events. The simplest possible version of a scheduler is a while loop.

## Setup Gemini client
We use Google's Gemini Pro model for this example, but you are free to use any model you want.

## Setup hyperliquid sdk 
We use the hyperliquid L1 network for this example, but you are free to use any app/chain you want.
Hyperliquid offers a great SDK for python, fast settlement times for trades, and friendly user interface.

In [28]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
load_dotenv()

# Configure Gemini client
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Configure model
model = genai.GenerativeModel('gemini-2.0-flash-exp')

# If you're getting an error here, ensure your environment is setup and that you have:
# 1. Installed the google-generativeai package: pip install google-generativeai
# 2. Set up your .env file with GOOGLE_API_KEY="your_api_key_here"

In [17]:
from hyperliquid.info import Info
from hyperliquid.utils import constants
import json

info = Info(constants.TESTNET_API_URL, skip_ws=True)

# This is an example address, when you start trading this should be replaced with your address
user_state = info.user_state("0xcd5051944f780a621ee62e39e493c489668acf4d") 
print(json.dumps(user_state, indent=2))

# if you're getting an error or no user_state here make sure your environment variables are set correctly and that all requirements are installed

{
  "marginSummary": {
    "accountValue": "326777.63864",
    "totalNtlPos": "45963.700635",
    "totalRawUsd": "353126.188565",
    "totalMarginUsed": "2314.174934"
  },
  "crossMarginSummary": {
    "accountValue": "326777.63864",
    "totalNtlPos": "45963.700635",
    "totalRawUsd": "353126.188565",
    "totalMarginUsed": "2314.174934"
  },
  "crossMaintenanceMarginUsed": "459.637003",
  "withdrawable": "324463.463706",
  "assetPositions": [
    {
      "type": "oneWay",
      "position": {
        "coin": "ATOM",
        "szi": "491.45",
        "leverage": {
          "type": "cross",
          "value": 20
        },
        "entryPx": "10.9725",
        "positionValue": "3288.046225",
        "unrealizedPnl": "-2104.3889",
        "returnOnEquity": "-7.80496696",
        "liquidationPx": null,
        "marginUsed": "164.402311",
        "maxLeverage": 50,
        "cumFunding": {
          "allTime": "-1619.741957",
          "sinceOpen": "-1619.741957",
          "sinceChange": 

## Define example identity of a user
This can be personalized for each individual user. We'll end up passing these fields to our prompts later, so that the same agent system can be used by many people with different goals and values.

To think about: what other fields might be useful here? Are there sources of data you could gather from users instead of asking these questions directly? Ex: If I gave you my wallet address, what info would be helpful there? How would you gather and analyze that?

In [18]:
identity_map = {
    "risk_level": "Moderate risk, willing to risk some money for the right investments but not chasing every new opportunity.",
    "token_preferences": "Likes ETH more than BTC, doesnt like SOL",
    "mission_statement": "Accumulate as much ETH as possible given the available funds.",
}

In [19]:
# The amount available to withdraw is approximately equal to the balance available to trade
available_balance = user_state.get('withdrawable') # denominated in USD
print(f"Available balance: {available_balance} USD")

Available balance: 324463.463706 USD


In [20]:
# We're also going to take a look at the current positions that are open, and format them for use with our models
positions = user_state.get('assetPositions')
positions_for_llm = ''
if positions:
    for position in positions:
        position = position.get('position')
        position_for_llm = f"Current {position.get('coin')} position: size {position.get('szi')} {position.get('coin')}, value of {position.get('positionValue')} usd, leverage {position.get('leverage').get('value')}, and unrealizedPnl {position.get('unrealizedPnl')} usd. The max leverage for this position is {position.get('maxLeverage')}.\n"
        positions_for_llm += position_for_llm
print(positions_for_llm)

Current ATOM position: size 491.45 ATOM, value of 3288.046225 usd, leverage 20, and unrealizedPnl -2104.3889 usd. The max leverage for this position is 50.
Current BTC position: size -0.15792 BTC, value of 15457.2096 usd, leverage 20, and unrealizedPnl -10476.697056 usd. The max leverage for this position is 50.
Current ETH position: size -2.8225 ETH, value of 9879.59675 usd, leverage 20, and unrealizedPnl -4210.7184 usd. The max leverage for this position is 50.
Current BNB position: size -14.872 BNB, value of 10514.35528 usd, leverage 20, and unrealizedPnl -5363.730047 usd. The max leverage for this position is 50.
Current GMT position: size 232.0 GMT, value of 35.53312 usd, leverage 2, and unrealizedPnl -39.516096 usd. The max leverage for this position is 50.
Current DYDX position: size -200.7 DYDX, value of 304.96365 usd, leverage 20, and unrealizedPnl 302.89644 usd. The max leverage for this position is 50.
Current APE position: size 2689.3 APE, value of 3398.19948 usd, leverage 

In [21]:
recent_headlines = ("*ROBINHOOD: CRYPTO TRADING VOLUMES OVER $30B, UP 600% YOY\n"
                   "*FED'S MUSALEM: TIME MAY BE APPROACHING TO SLOW OR PAUSE RATE CUTS\n")

chat_summary = "Bob is happy with the Ethereum roadmap and has been hearing more people talk about it.\n"

chat_and_data_summary = chat_summary + recent_headlines

## Create our prompts
We do this with markdown because LLMs respond well to this format.
This is structured as a system prompt that remains fixed for all users, and a user prompt unique to each user.

In [22]:
from pydantic import BaseModel, Field
from typing import Literal

# This class will be used to submit the trades
class Position(BaseModel):
    market: str = Field(..., description="The asset to trade")
    direction: Literal["long", "short"] = Field(
        ...,
        description="The direction to trade",
    )
    size: float = Field(
        ...,
        description="The size of the trade denominated in USD. It should be greater than 10.",
    )
    reasoning: list[str] = Field(
        default_factory=list,
        description="The reasoning for the decision",
    )
    leverage: int | None = Field(None, description="Optional leverage multiplier")


class PositionReasoning(BaseModel):
    positions_to_maintain: list[Position] = Field(
        default_factory=list,
        description="Positions to maintain without changes",
    )
    positions_to_modify: list[Position] = Field(
        default_factory=list,
        description="Positions to modify or close",
    )
    positions_to_open: list[Position] = Field(
        default_factory=list,
        description="Positions to open",
    )

In [33]:
# First part - Analyze current positions
positions_prompt = (
    "Analyze these trading positions and return which ones to maintain as a JSON array.\n"
    f"{positions_for_llm}\n"
    "Return EXACTLY in this format:\n"
    "{\n"
    '  "positions_to_maintain": [\n'
    "    {\n"
    '      "market": "DYDX",\n'
    '      "direction": "short",\n'
    '      "size": 100.0,\n'
    '      "reasoning": ["Reason 1", "Reason 2"],\n'
    '      "leverage": 20\n'
    "    }\n"
    "  ]\n"
    "}"
)

try:
    response_positions = model.generate_content(positions_prompt)
    positions_result = response_positions.text
    print("Positions analysis:", positions_result)
except Exception as e:
    print(f"Error analyzing positions: {e}")

Positions analysis: ```json
{
  "positions_to_maintain": [
    {
      "market": "DYDX",
      "direction": "short",
       "size": 200.7,
      "reasoning": ["Only position with a profit.", "High leverage is being used."],
      "leverage": 20
    }
  ]
}
```



In [34]:
# Second part - Analyze which positions need modifications
modifications_prompt = (
    "Based on current market conditions and these positions:\n"
    f"{positions_for_llm}\n"
    f"Recent info: {chat_and_data_summary}\n"
    "Return EXACTLY in this format:\n"
    "{\n"
    '  "positions_to_modify": [\n'
    "    {\n"
    '      "market": "BTC",\n'
    '      "direction": "short",\n'
    '      "size": 100.0,\n'
    '      "reasoning": ["Reason for modification"],\n'
    '      "leverage": 20\n'
    "    }\n"
    "  ]\n"
    "}"
)

try:
    response_modifications = model.generate_content(modifications_prompt)
    modifications_result = response_modifications.text
    print("Modifications analysis:", modifications_result)
except Exception as e:
    print(f"Error analyzing modifications: {e}")

Modifications analysis: ```json
{
  "positions_to_modify": [
    {
      "market": "BTC",
      "direction": "short",
      "size": 100.0,
       "reasoning": ["The Fed may pause or slow rate cuts, potentially reducing liquidity and risk appetite for assets like Bitcoin. Robinhood seeing increased volume implies volatility and uncertainty."],
       "leverage": 20
    },
    {
      "market": "ETH",
      "direction": "short",
      "size": 25.0,
       "reasoning": ["While Bob is happy with the ETH roadmap, the overall macro environment and increased trading volume (implying increased market uncertainity) makes this a higher risk position and a slight reduction is warranted."],
      "leverage": 20
    },
     {
      "market": "APE",
      "direction": "short",
      "size": 25.0,
       "reasoning": ["The unrealized loss of APE is very high compared to its position size, so reducing risk is wise."],
      "leverage": 20
    }
  ]
}
```



In [35]:
# Third part - Suggest new positions
new_positions_prompt = (
    f"Given ${available_balance} available balance and this recent information:\n"
    f"{chat_and_data_summary}\n"
    "Return EXACTLY in this format:\n"
    "{\n"
    '  "positions_to_open": [\n'
    "    {\n"
    '      "market": "ETH",\n'
    '      "direction": "long",\n'
    '      "size": 1000.0,\n'
    '      "reasoning": ["Reason 1", "Reason 2"],\n'
    '      "leverage": 10\n'
    "    }\n"
    "  ]\n"
    "}"
)

try:
    response_new = model.generate_content(new_positions_prompt)
    new_positions_result = response_new.text
    print("New positions suggestions:", new_positions_result)
except Exception as e:
    print(f"Error suggesting new positions: {e}")

# Combine all results with better error handling
try:
    import json
    import re
    
    def extract_json(text):
        # Find JSON content between triple backticks if present
        json_match = re.search(r'```json\s*(\{[\s\S]*?\})\s*```', text)
        if json_match:
            return json_match.group(1)
        # If no backticks, find JSON between curly braces
        json_match = re.search(r'(\{[\s\S]*\})', text)
        if json_match:
            return json_match.group(1)
        return "{}"

    # Extract and parse JSON from each response
    positions_json = json.loads(extract_json(positions_result)) if 'positions_result' in locals() else {"positions_to_maintain": []}
    modifications_json = json.loads(extract_json(modifications_result)) if 'modifications_result' in locals() else {"positions_to_modify": []}
    new_positions_json = json.loads(extract_json(new_positions_result)) if 'new_positions_result' in locals() else {"positions_to_open": []}
    
    # Combine results
    final_result = {
        "positions_to_maintain": positions_json.get("positions_to_maintain", []),
        "positions_to_modify": modifications_json.get("positions_to_modify", []),
        "positions_to_open": new_positions_json.get("positions_to_open", [])
    }
    
    # Validate with Pydantic model
    result = PositionReasoning.model_validate(final_result)
    print("\nFinal combined analysis:")
    print(result.model_dump())
except Exception as e:
    print(f"Error combining results: {e}")
    print("\nRaw responses:")
    print("Positions:", positions_result if 'positions_result' in locals() else "No data")
    print("Modifications:", modifications_result if 'modifications_result' in locals() else "No data")
    print("New positions:", new_positions_result if 'new_positions_result' in locals() else "No data")

New positions suggestions: ```json
{
  "positions_to_open": [
    {
      "market": "ETH",
      "direction": "long",
      "size": 1000.0,
      "reasoning": ["Positive sentiment around the Ethereum roadmap suggests potential price appreciation.", "Robinhood's significant increase in crypto trading volume, particularly in the altcoin space where ETH is a major player, indicates growing market interest and potential upward momentum."],
       "leverage": 10
    }
  ]
}
```


Final combined analysis:
{'positions_to_maintain': [{'market': 'DYDX', 'direction': 'short', 'size': 200.7, 'reasoning': ['Only position with a profit.', 'High leverage is being used.'], 'leverage': 20}], 'positions_to_modify': [{'market': 'BTC', 'direction': 'short', 'size': 100.0, 'reasoning': ['The Fed may pause or slow rate cuts, potentially reducing liquidity and risk appetite for assets like Bitcoin. Robinhood seeing increased volume implies volatility and uncertainty.'], 'leverage': 20}, {'market': 'ETH', 'd

## Submit our trades (Incomplete example for testing)

For positions_to_maintain there is no action required, but it's helpful to log and explain the "reasoning" to users.
positions_to_open and positions_to_modify both require submitting orders onchain. 

If you wish to start trading before updates to this notebook are available, you can refer to the [hyperliquid-python-sdk.](https://github.com/hyperliquid-dex/hyperliquid-python-sdk)